In [1]:
#%% Importar librerias
import pyodbc
import pandas as pd

In [2]:
#%%RUTA DEL ARCHIVO
ruta = r"C:\Users\bgacitua\Desktop\Reportes Corrección de Marcas\Reportes_marcas_rflex\reporte_turnos_sin_marcas_20250812075544.xlsx"



In [3]:
#%% Configuración de conexión
SERVER = '192.9.200.84' 
DATABASE = 'MorphoManager'  
USERNAME = 'rrhh_morpho'  
PASSWORD = 'PQ90@ZT*'  

#%% Cadena de conexión para SQL Server
try:
    connection_string = (
        'DRIVER={ODBC Driver 17 for SQL Server};'
        f'SERVER={SERVER};'
        f'DATABASE={DATABASE};'
        f'UID={USERNAME};'
        f'PWD={PASSWORD};'
        'TrustServerCertificate=yes;'  # Para evitar problemas con certificados SSL
    )
    
    cnxn = pyodbc.connect(connection_string)
    print("¡Conexión a SQL Server establecida exitosamente! Lista para múltiples consultas.")

except pyodbc.Error as ex:
    sqlstate = ex.args[0]
    print(f"Error al conectar a SQL Server: {ex}")
    if sqlstate == '28000':
        print("Causa probable: Error de autenticación (usuario o contraseña incorrectos).")
    elif sqlstate == '08001':
        print("Causa probable: El servidor SQL Server no está accesible, el nombre del servidor es incorrecto, o el puerto (1433 por defecto) está bloqueado por un firewall.")
    else:
        print("Consulta el mensaje de error para más detalles.")


¡Conexión a SQL Server establecida exitosamente! Lista para múltiples consultas.


In [4]:
# %%
import datetime
fecha_actual_py = datetime.datetime.now()
fecha_inicio_sql = fecha_actual_py.replace(day=1).strftime('%Y-%m-%d')  # Primer día del mes actual
fecha_fin_sql = fecha_actual_py.strftime('%Y-%m-%d')  # Fecha actual

print(f"Fecha inicio: {fecha_inicio_sql}")
print(f"Fecha fin: {fecha_fin_sql}")
#%%   
# --- 3. Bloque para Consultas Iterativas y Transformación a DataFrame ---
# Puedes cambiar la consulta SQL aquí cada vez que quieras probar una nueva.
sql_query = f"""
-- Versión optimizada con rango de fechas automático
-- Las fechas se calculan automáticamente desde Python
DECLARE @fecha_inicio DATE = '{fecha_inicio_sql}';
DECLARE @fecha_fin DATE = '{fecha_fin_sql}';

WITH fechas_rango AS (
    -- Generar secuencia de fechas entre inicio y fin
    SELECT @fecha_inicio AS [FECHA_SANTIAGO]
    UNION ALL
    SELECT DATEADD(DAY, 1, [FECHA_SANTIAGO])
    FROM fechas_rango
    WHERE DATEADD(DAY, 1, [FECHA_SANTIAGO]) <= @fecha_fin
),
todos_trabajadores AS (
    -- Obtener todos los trabajadores activos
    SELECT DISTINCT 
        u.[ID] AS [USER_ID],
        u.[FIRSTNAME],
        u.[LASTNAME],
        u.[EMPLOYEEID]
    FROM [MorphoManager].[dbo].[User_] AS u
    WHERE u.[EMPLOYEEID] IS NOT NULL 
    AND u.[EMPLOYEEID] != ''  -- Filtrar empleados válidos
),
combinaciones_trabajador_fecha AS (
    -- Crear matriz completa trabajador x fecha
    SELECT 
        t.[USER_ID],
        t.[FIRSTNAME],
        t.[LASTNAME], 
        t.[EMPLOYEEID],
        f.[FECHA_SANTIAGO]
    FROM todos_trabajadores t
    CROSS JOIN fechas_rango f
)
SELECT 
    -- Información del trabajador
    ctf.[FIRSTNAME],
    ctf.[LASTNAME],
    ctf.[EMPLOYEEID],
    ctf.[FECHA_SANTIAGO],
    
    -- Conteo de registros por tipo (0 si no hay marcas)
    COALESCE(SUM(CASE WHEN m.[FUNCTIONKEYTEXT] = 'IN' THEN 1 ELSE 0 END), 0) AS [KEY_IN],
    COALESCE(SUM(CASE WHEN m.[FUNCTIONKEYTEXT] = 'OUT' THEN 1 ELSE 0 END), 0) AS [KEY_OUT],
    COALESCE(SUM(CASE WHEN m.[FUNCTIONKEYTEXT] = 'No Key' THEN 1 ELSE 0 END), 0) AS [NO_KEY],
    
    -- Estado de asistencia
    CASE 
        WHEN COUNT(m.[ID]) = 0 THEN 'Ausente'
        WHEN COUNT(m.[ID]) >= 1 THEN 'Asiste'
    END AS [ESTADO_ASISTENCIA],
    
    -- Información adicional
    COALESCE(COUNT(DISTINCT bd.[NAME_]), 0) AS [CANTIDAD_DISPOSITIVOS],
    COALESCE(COUNT(m.[ID]), 0) AS [TOTAL_REGISTROS],
    
    -- Información de horarios (opcional)
    COALESCE(MIN(CAST(m.[LOGDATETIME] AT TIME ZONE 'UTC' AT TIME ZONE 'Pacific SA Standard Time' AS TIME)), '00:00:00') AS [PRIMERA_MARCA],
    COALESCE(MAX(CAST(m.[LOGDATETIME] AT TIME ZONE 'UTC' AT TIME ZONE 'Pacific SA Standard Time' AS TIME)), '00:00:00') AS [ULTIMA_MARCA]
    
FROM combinaciones_trabajador_fecha ctf
LEFT JOIN [MorphoManager].[dbo].[AccessLog] AS m 
    ON ctf.[USER_ID] = m.[USERID]
    AND ctf.[FECHA_SANTIAGO] = CAST(m.[LOGDATETIME] AT TIME ZONE 'UTC' AT TIME ZONE 'Pacific SA Standard Time' AS DATE)
    AND CAST(m.[LOGDATETIME] AT TIME ZONE 'UTC' AT TIME ZONE 'Pacific SA Standard Time' AS DATE) BETWEEN @fecha_inicio AND @fecha_fin
LEFT JOIN [MorphoManager].[dbo].[BiometricDevice] AS bd 
    ON m.[MORPHOACCESSID] = bd.[ID]
    
GROUP BY 
    ctf.[FIRSTNAME],
    ctf.[LASTNAME],
    ctf.[EMPLOYEEID],
    ctf.[FECHA_SANTIAGO],
    ctf.[USER_ID]
    
ORDER BY 
    ctf.[LASTNAME], ctf.[FIRSTNAME], ctf.[FECHA_SANTIAGO] DESC
OPTION (MAXRECURSION 366);  -- Permite hasta 366 días en la recursión
"""

Fecha inicio: 2025-08-01
Fecha fin: 2025-08-12


In [5]:
#%% TRANSFORMAMOS CONSULTA EN DATAFRAME PARA TRABAJAR CON PANDAS

try:
    # Utiliza pandas.read_sql para ejecutar la consulta con la conexión ya abierta
    df_resultado = pd.read_sql(sql_query, cnxn)

    print("\n--- ¡Consulta ejecutada y DataFrame creado! ---")
    #print(f"Consulta ejecutada:\n{sql_query.strip()}") # strip() para limpiar espacios al inicio/fin

    print("\nPrimeras 5 filas del DataFrame:")
    print(df_resultado.head())

    print(f"\nNúmero de filas y columnas: {df_resultado.shape}")
    print("Columnas y tipos de datos:")
    df_resultado.info() # df.info() no retorna, solo imprime, por eso no se usa print()

except pyodbc.Error as ex:
    # Manejo de errores de consulta (ej. tabla no existe, sintaxis SQL incorrecta)
    print(f"Error al ejecutar la consulta SQL: {ex}")
    print("Por favor, revisa tu consulta SQL o si la tabla/columna existe y tienes permisos.")
except Exception as e:
    # Manejo de cualquier otro error inesperado
    print(f"Ocurrió un error inesperado durante el procesamiento: {e}")

# NOTA: La conexión NO se cierra aquí. Se mantiene abierta para la siguiente consulta.

#%% SEPARAR DV DEL RUT
# Paso 1: Identificar las filas que contienen el delimitador "-"
tiene_delimitador = df_resultado['EMPLOYEEID'].astype(str).str.contains('-')

# Paso 2: Crear la nueva columna con los valores después del delimitador
df_resultado['ID_ADICIONAL'] = df_resultado['EMPLOYEEID'].astype(str).str.split('-').str[1].fillna('')

# Paso 3: Actualizar la columna EMPLOYEEID para mantener solo la parte antes del delimitador
df_resultado.loc[tiene_delimitador, 'EMPLOYEEID'] = df_resultado.loc[tiene_delimitador, 'EMPLOYEEID'].astype(str).str.split('-').str[0]

try:
    df_resultado['EMPLOYEEID'] = pd.to_numeric(df_resultado['EMPLOYEEID'], errors='coerce')
    # errors='coerce' convertirá los valores que no puedan ser convertidos a números en NaN
    print("\n'EMPLOYEEID' se convirtió a tipo numérico (NaN si no pudo convertir).")
except Exception as e:
    print(f"\nAdvertencia: No se pudo convertir 'EMPLOYEEID' a numérico debido a: {e}")

# Mostrar el DataFrame después de la transformación
print("\nDataFrame después de la transformación:")
print(df_resultado[['EMPLOYEEID', 'ID_ADICIONAL']].head(5))

df_resultado.info()
#%%CREAR COLUMNA DE BUSQUEDA RUT + FECHA

df_resultado['EMPLOYEEID_STR'] = df_resultado['EMPLOYEEID'].astype(str) 
df_resultado['EMPLOYEEID_STR'] = df_resultado['EMPLOYEEID_STR'].str.replace(r'\.0$', '', regex=True)
#df_resultado['EMPLOYEEID'] = df_resultado['EMPLOYEEID'].astype(int) 
df_resultado['FECHA_SANTIAGO_STR'] = df_resultado['FECHA_SANTIAGO'].astype(str)
df_resultado.info()
df_resultado['BusquedaRUT'] = df_resultado['EMPLOYEEID_STR'] + ";" + df_resultado['FECHA_SANTIAGO_STR']

print(df_resultado)

C:\Users\bgacitua\AppData\Local\Temp\ipykernel_7364\470403224.py:5: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_resultado = pd.read_sql(sql_query, cnxn)



--- ¡Consulta ejecutada y DataFrame creado! ---

Primeras 5 filas del DataFrame:
   FIRSTNAME LASTNAME  EMPLOYEEID FECHA_SANTIAGO  KEY_IN  KEY_OUT  NO_KEY  \
0  Yessenia   Abanto   25022430-3     2025-08-12       0        0       0   
1  Yessenia   Abanto   25022430-3     2025-08-11       0        0       0   
2  Yessenia   Abanto   25022430-3     2025-08-10       0        0       0   
3  Yessenia   Abanto   25022430-3     2025-08-09       0        0       0   
4  Yessenia   Abanto   25022430-3     2025-08-08       0        0       7   

  ESTADO_ASISTENCIA  CANTIDAD_DISPOSITIVOS  TOTAL_REGISTROS PRIMERA_MARCA  \
0           Ausente                      0                0      00:00:00   
1           Ausente                      0                0      00:00:00   
2           Ausente                      0                0      00:00:00   
3           Ausente                      0                0      00:00:00   
4            Asiste                      5                7      11:20

In [6]:
#%%Info del Dataframe SQL
df_resultado.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9012 entries, 0 to 9011
Data columns (total 16 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   FIRSTNAME              9012 non-null   object 
 1   LASTNAME               9012 non-null   object 
 2   EMPLOYEEID             8700 non-null   float64
 3   FECHA_SANTIAGO         9012 non-null   object 
 4   KEY_IN                 9012 non-null   int64  
 5   KEY_OUT                9012 non-null   int64  
 6   NO_KEY                 9012 non-null   int64  
 7   ESTADO_ASISTENCIA      9012 non-null   object 
 8   CANTIDAD_DISPOSITIVOS  9012 non-null   int64  
 9   TOTAL_REGISTROS        9012 non-null   int64  
 10  PRIMERA_MARCA          9012 non-null   object 
 11  ULTIMA_MARCA           9012 non-null   object 
 12  ID_ADICIONAL           9012 non-null   object 
 13  EMPLOYEEID_STR         9012 non-null   object 
 14  FECHA_SANTIAGO_STR     9012 non-null   object 
 15  Busq

In [7]:
#%%IMPORTAR REPORTE RFLEX A TRAVÉS DE LA RUTA
#SOLO SE DEBE COPIAR LA RUTA Y EJECUTAR TODO EL CÓDIGO

ruta_excel_ruts = ruta
nombre_columna_ruts_excel = 'RUT'

# --- 2. Leer el listado de RUTs desde el archivo Excel ---
try:
    df_ruts_excel = pd.read_excel(ruta_excel_ruts)
    
    print(f"\nArchivo Excel leído exitosamente desde '{ruta_excel_ruts}'")
    print(f"Dimensiones del archivo: {df_ruts_excel.shape}")
    print("\nPrimeras filas del archivo original:")
    print(df_ruts_excel.head())
    
    # --- 3. Procesar la columna RUT para separar RUT y DV ---
    print(f"\nProcesando separación de RUT y DV...")
    
    # Convertir la columna RUT a string para poder trabajar con ella
    df_ruts_excel[nombre_columna_ruts_excel] = df_ruts_excel[nombre_columna_ruts_excel].astype(str)
    
    # Verificar si contiene el delimitador "-"
    tiene_delimitador = df_ruts_excel[nombre_columna_ruts_excel].str.contains('-', na=False)
    
    if tiene_delimitador.any():
        # Separar RUT y DV usando el delimitador "-"
        rut_split = df_ruts_excel[nombre_columna_ruts_excel].str.split('-', expand=True)
        
        # Actualizar la columna RUT con solo la parte numérica
        df_ruts_excel[nombre_columna_ruts_excel] = rut_split[0]
        
        # Crear la columna DV con el dígito verificador
        df_ruts_excel['DV'] = rut_split[1].fillna('')
        
        print(f"✅ Separación completada:")
        print(f"   - Registros con delimitador procesados: {tiene_delimitador.sum()}")
        print(f"   - RUT: contiene solo números")
        print(f"   - DV: nueva columna creada con dígitos verificadores")
    else:
        # Si no hay delimitador, crear columna DV vacía
        df_ruts_excel['DV'] = ''
        print("ℹ️  No se encontraron delimitadores '-' en la columna RUT")
    
    # Mostrar resultado después del procesamiento
    print("\nPrimeras filas después del procesamiento:")
    print(df_ruts_excel[['RUT', 'DV']].head(10))
    
    # Convertir RUT a entero para compatibilidad
    try:
        df_ruts_excel['RUT'] = pd.to_numeric(df_ruts_excel['RUT'], errors='coerce')
        print(f"✅ Columna RUT convertida a tipo numérico")
    except Exception as e:
        print(f"⚠️  Advertencia al convertir RUT a numérico: {e}")

except FileNotFoundError:
    print(f"❌ Error: El archivo Excel '{ruta_excel_ruts}' no se encontró.")
    print("   Asegúrate de la ruta y el nombre del archivo.")
    df_ruts_excel = pd.DataFrame()  # DataFrame vacío para evitar errores
except KeyError:
    print(f"❌ Error: La columna '{nombre_columna_ruts_excel}' no se encontró en el archivo Excel.")
    print("   Revisa el encabezado del archivo.")
    df_ruts_excel = pd.DataFrame()
except Exception as e:
    print(f"❌ Error inesperado al leer el archivo Excel: {e}")
    df_ruts_excel = pd.DataFrame()


Archivo Excel leído exitosamente desde 'C:\Users\bgacitua\Desktop\Reportes Corrección de Marcas\Reportes_marcas_rflex\reporte_turnos_sin_marcas_20250812075544.xlsx'
Dimensiones del archivo: (85, 10)

Primeras filas del archivo original:
          RUT                 Nombre              Unidad  \
0  14509971-4           Ruben Diaz .    Chofer y Peoneta   
1   9252524-4  Victor Acosta Alegria            Customer   
2  11484287-7    Ivonne Castro Ibeas           Comprador   
3  12634159-8  Rodrigo Arenas Guerra  Ayudante De Bodega   
4  26820457-1  Gerardo Perez Vasquez           Asistente   

                                    Puesto       Fecha Tipo de turno  \
0  Bodega Recepción, Despacho y Transporte  2025-08-08             D   
1        Administración Gral De Producción  2025-08-08             D   
2     Compras Nacionales e Internacionales  2025-08-08             D   
3  Bodega Recepción, Despacho y Transporte  2025-08-08             D   
4        Administración Gral De Producció

In [8]:
#%% Procesar columnas y crear clave de búsqueda
# Convertir RUT a string para crear la clave de búsqueda
df_ruts_excel['RUT'] = df_ruts_excel['RUT'].astype(str)

print("Información del DataFrame después del procesamiento:")
df_ruts_excel.info()

# Crear columna de búsqueda RUT + Fecha
df_ruts_excel['Busqueda'] = df_ruts_excel['RUT'] + ";" + df_ruts_excel["Fecha"]

print("\nDataFrame con columna de búsqueda creada:")
print(df_ruts_excel[['RUT', 'DV', 'Fecha', 'Busqueda']].head())
print(f"\nTotal de registros: {len(df_ruts_excel)}")

Información del DataFrame después del procesamiento:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 85 entries, 0 to 84
Data columns (total 11 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   RUT                 85 non-null     object
 1   Nombre              85 non-null     object
 2   Unidad              85 non-null     object
 3   Puesto              85 non-null     object
 4   Fecha               85 non-null     object
 5   Tipo de turno       85 non-null     object
 6   Entrada programada  85 non-null     object
 7   Salida programada   85 non-null     object
 8   Marca de entrada    85 non-null     object
 9   Marca de salida     85 non-null     object
 10  DV                  85 non-null     object
dtypes: object(11)
memory usage: 7.4+ KB

DataFrame con columna de búsqueda creada:
        RUT DV       Fecha             Busqueda
0  14509971  4  2025-08-08  14509971;2025-08-08
1   9252524  4  2025-08-08   9252524;202

In [9]:
#%%IMPORTAR NUMPY
import numpy as np

#%%

df_ruts_excel['Marca de entrada'] = df_ruts_excel['Marca de entrada'].replace("--:--", np.nan)
df_ruts_excel['Marca de salida'] = df_ruts_excel['Marca de salida'].replace("--:--", np.nan)

df_ruts_excel.isna()
print(df_ruts_excel)

#%%Traer Key de Marca

# Columnas a incorporar al nuevo DF.
columnas_a_traer = ['KEY_IN', 'KEY_OUT', 'NO_KEY', 'PRIMERA_MARCA', 'ULTIMA_MARCA']

# Realizar el merge
# left_on: la columna de df_ruts_excel (izquierda) para la coincidencia
# right_on: la columna de df_resultado (derecha) para la coincidencia
# how='left': asegura que todas las filas de df_ruts_excel se mantengan.
# Si no hay coincidencia en df_resultado, las nuevas columnas tendrán NaN.

df_ruts_excel_con_datos = pd.merge(
    df_ruts_excel,           # DataFrame de la izquierda (el que quieres enriquecer)
    df_resultado[columnas_a_traer + ['BusquedaRUT']], # DataFrame de la derecha (solo las columnas que necesitas + la de búsqueda)
    left_on='Busqueda',      # Columna para buscar en df_ruts_excel
    right_on='BusquedaRUT',  # Columna para buscar en df_resultado
    how='left'               # Tipo de merge: 'left' para mantener todas las filas de df_ruts_excel
)

df_ruts_excel_con_datos = df_ruts_excel_con_datos.drop(columns=['BusquedaRUT'])
print(df_ruts_excel_con_datos)

         RUT                 Nombre              Unidad  \
0   14509971           Ruben Diaz .    Chofer y Peoneta   
1    9252524  Victor Acosta Alegria            Customer   
2   11484287    Ivonne Castro Ibeas           Comprador   
3   12634159  Rodrigo Arenas Guerra  Ayudante De Bodega   
4   26820457  Gerardo Perez Vasquez           Asistente   
..       ...                    ...                 ...   
80   9689246  Luis Gonzalez Vasquez            Operario   
81   9262185      Marco Pinto Palma            Operario   
82  13702299   Manuel Cortés Zuñiga            Jefatura   
83   8275564      Marcela Rios Seco           Asistente   
84  21530953    María Arellano Curi           Asistente   

                                     Puesto       Fecha Tipo de turno  \
0   Bodega Recepción, Despacho y Transporte  2025-08-08             D   
1         Administración Gral De Producción  2025-08-08             D   
2      Compras Nacionales e Internacionales  2025-08-08             D   

In [10]:
#%%CRUCE DE DATOS

df_rellenar = df_ruts_excel_con_datos.copy()

#Diccionario para marcas de entrada
mapa_entrada = df_resultado.set_index('BusquedaRUT')['PRIMERA_MARCA'].to_dict()

#Diccionario para marcas de salida
mapa_salida = df_resultado.set_index('BusquedaRUT')['ULTIMA_MARCA'].to_dict()

print("\nMapa de Entrada:\n", mapa_entrada) 
print("\nMapa de Salida:\n", mapa_salida)  

#Buscar Marcas de Entrada
valores_entrada_buscados = df_rellenar['Busqueda'].map(mapa_entrada)

#Reemplazar valores de entrada
df_rellenar['Marca de entrada Generados'] = df_rellenar['Marca de entrada'].fillna(valores_entrada_buscados)

# Procesar Marca de salida
valores_salida_buscados = df_rellenar['Busqueda'].map(mapa_salida)
df_rellenar['Marca de salida Generados'] = df_rellenar['Marca de salida'].fillna(valores_salida_buscados)

print("\nDataFrame Principal (después del BUSCARV condicional):\n", df_rellenar)

print("\nConteo de NaNs después del proceso:")
print(df_rellenar.isna().sum())

#df_rellenar["Marca de entrada"] = df_rellenar["Marca de entrada"].str.strip()

#df_rellenar["Marca de entrada"] = df_rellenar["Marca de entrada"].replace("00:00:00", "Sin marca")

#print(df_rellenar)


Mapa de Entrada:
 {'25022430;2025-08-12': datetime.time(0, 0), '25022430;2025-08-11': datetime.time(0, 0), '25022430;2025-08-10': datetime.time(0, 0), '25022430;2025-08-09': datetime.time(0, 0), '25022430;2025-08-08': datetime.time(11, 20, 8), '25022430;2025-08-07': datetime.time(11, 42, 50), '25022430;2025-08-06': datetime.time(12, 5, 40), '25022430;2025-08-05': datetime.time(11, 39, 43), '25022430;2025-08-04': datetime.time(11, 46, 51), '25022430;2025-08-03': datetime.time(0, 0), '25022430;2025-08-02': datetime.time(7, 28, 28), '25022430;2025-08-01': datetime.time(10, 55, 30), '12357791;2025-08-12': datetime.time(6, 48, 18), '12357791;2025-08-11': datetime.time(6, 48, 16), '12357791;2025-08-10': datetime.time(0, 0), '12357791;2025-08-09': datetime.time(0, 0), '12357791;2025-08-08': datetime.time(0, 54, 8), '12357791;2025-08-07': datetime.time(0, 49, 2), '12357791;2025-08-06': datetime.time(0, 56, 24), '12357791;2025-08-05': datetime.time(0, 54, 24), '12357791;2025-08-04': datetime.t

In [11]:
#%%Manipulación del Df_rellenar para que evalúe si la marca es mayor a la hora de entrada o menor a la hora de salida

# Convertir las columnas de tiempo a datetime para poder hacer comparaciones
# Nota: Las columnas programadas están en formato HH:MM, las marcas en HH:MM:SS
df_rellenar['Entrada_programada_dt'] = pd.to_datetime(df_rellenar['Entrada programada'], format='%H:%M', errors='coerce').dt.time
df_rellenar['Salida_programada_dt'] = pd.to_datetime(df_rellenar['Salida programada'], format='%H:%M', errors='coerce').dt.time
df_rellenar['PRIMERA_MARCA_dt'] = pd.to_datetime(df_rellenar['PRIMERA_MARCA'], format='%H:%M:%S', errors='coerce').dt.time
df_rellenar['ULTIMA_MARCA_dt'] = pd.to_datetime(df_rellenar['ULTIMA_MARCA'], format='%H:%M:%S', errors='coerce').dt.time

# Función para calcular diferencia en horas entre dos tiempos
def diferencia_horas(tiempo1, tiempo2):
    if pd.isna(tiempo1) or pd.isna(tiempo2):
        return np.nan
    
    # Convertir time a datetime para poder calcular diferencias
    fecha_base = pd.Timestamp('2000-01-01')
    dt1 = pd.Timestamp.combine(fecha_base.date(), tiempo1)
    dt2 = pd.Timestamp.combine(fecha_base.date(), tiempo2)
    
    # Calcular diferencia absoluta en horas
    diferencia = abs((dt2 - dt1).total_seconds() / 3600)
    return diferencia

# Calcular las diferencias
df_rellenar['Diferencia_entrada'] = df_rellenar.apply(
    lambda row: diferencia_horas(row['Entrada_programada_dt'], row['PRIMERA_MARCA_dt']), axis=1
)

df_rellenar['Diferencia_salida'] = df_rellenar.apply(
    lambda row: diferencia_horas(row['Salida_programada_dt'], row['ULTIMA_MARCA_dt']), axis=1
)

# Crear la columna "Observación de turno"
df_rellenar['Observación de turno'] = np.where(
    (df_rellenar['Diferencia_entrada'] >= 2) & (df_rellenar['Diferencia_salida'] >= 2),
    'Observación de turno',
    None
)

# Limpiar columnas auxiliares (opcional)
df_rellenar = df_rellenar.drop(columns=[
    'Entrada_programada_dt', 'Salida_programada_dt', 'PRIMERA_MARCA_dt', 'ULTIMA_MARCA_dt', 'Diferencia_entrada', 'Diferencia_salida'])

print("\nDataFrame después de agregar la columna 'Observación de turno':")
print(df_rellenar[['RUT', 'Fecha', 'Entrada programada', 'PRIMERA_MARCA', 'Salida programada', 'ULTIMA_MARCA', 'Observación de turno']].head(10))

print(f"\nRegistros con observación de turno: {df_rellenar['Observación de turno'].notna().sum()}")


DataFrame después de agregar la columna 'Observación de turno':
        RUT       Fecha Entrada programada PRIMERA_MARCA Salida programada  \
0  14509971  2025-08-08              07:50      06:34:03             15:30   
1   9252524  2025-08-08              07:50      07:45:40             15:30   
2  11484287  2025-08-08              07:50      07:10:36             14:30   
3  12634159  2025-08-08              07:50      06:57:57             15:30   
4  26820457  2025-08-08              07:50      07:10:51             15:30   
5  12906586  2025-08-09              14:00      13:30:30             22:00   
6  16914208  2025-08-09              07:00      06:54:03             13:00   
7  11524484  2025-08-08              07:50      09:02:47             15:30   
8   6986640  2025-08-08              07:50      06:28:35             15:30   
9  24035160  2025-08-08              07:50      07:34:47             15:30   

  ULTIMA_MARCA Observación de turno  
0     15:24:36                 None  


In [12]:
display(df_rellenar)

,RUT,Nombre,Unidad,Puesto,Fecha,Tipo de turno,Entrada programada,Salida programada,Marca de entrada,Marca de salida,DV,Busqueda,KEY_IN,KEY_OUT,NO_KEY,PRIMERA_MARCA,ULTIMA_MARCA,Marca de entrada Generados,Marca de salida Generados,Observación de turno
0,14509971,Ruben Diaz .,Chofer y Peoneta,"Bodega Recepción, Despacho y Transporte",2025-08-08,D,07:50,15:30,NaN,15:23:41,4,14509971;2025-08-08,0,2,6,06:34:03,15:24:36,06:34:03,15:23:41,None
1,9252524,Victor Acosta Alegria,Customer,Administración Gral De Producción,2025-08-08,D,07:50,15:30,NaN,17:37:52,4,9252524;2025-08-08,0,0,9,07:45:40,17:39:38,07:45:40,17:37:52,None
2,11484287,Ivonne Castro Ibeas,Comprador,Compras Nacionales e Internacionales,2025-08-08,D,07:50,14:30,NaN,14:56:58,7,11484287;2025-08-08,0,0,7,07:10:36,15:00:22,07:10:36,14:56:58,None
3,12634159,Rodrigo Arenas Guerra,Ayudante De Bodega,"Bodega Recepción, Despacho y Transporte",2025-08-08,D,07:50,15:30,NaN,15:20:04,8,12634159;2025-08-08,0,1,10,06:57:57,15:20:38,06:57:57,15:20:04,None
4,26820457,Gerardo Perez Vasquez,Asistente,Administración Gral De Producción,2025-08-08,D,07:50,15:30,NaN,14:30:31,1,26820457;2025-08-08,0,0,15,07:10:51,16:06:15,07:10:51,14:30:31,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
80,9689246,Luis Gonzalez Vasquez,Operario,Planta de Muestras,2025-08-08,D,07:50,15:30,NaN,NaN,2,9689246;2025-08-08,1,1,7,07:26:37,15:28:53,07:26:37,15:28:53,None
81,9262185,Marco Pinto Palma,Operario,Planta de Muestras,2025-08-08,D,07:50,15:30,NaN,NaN,5,9262185;2025-08-08,1,1,5,06:52:07,15:22:15,06:52:07,15:22:15,None
82,13702299,Manuel Cortés Zuñiga,Jefatura,Ingeniería y Medio Ambiente,2025-08-08,D,07:50,15:30,NaN,NaN,0,13702299;2025-08-08,0,0,14,08:33:23,15:09:55,08:33:23,15:09:55,None
83,8275564,Marcela Rios Seco,Asistente,Administación y Lavandería,2025-08-08,L4,10:30,16:00,NaN,NaN,0,8275564;2025-08-08,1,1,6,10:24:02,18:31:41,10:24:02,18:31:41,None


In [13]:
# Analizar el formato de las columnas de entrada y salida programada
print("Muestra de las columnas de tiempo:")
print(df_rellenar[['Entrada programada', 'Salida programada', 'PRIMERA_MARCA', 'ULTIMA_MARCA']].head(10))
print("\nTipos de datos:")
print(df_rellenar[['Entrada programada', 'Salida programada', 'PRIMERA_MARCA', 'ULTIMA_MARCA']].dtypes)
print("\nEjemplos únicos de 'Entrada programada':")
print(df_rellenar['Entrada programada'].unique()[:10])
print("\nEjemplos únicos de 'Salida programada':")
print(df_rellenar['Salida programada'].unique()[:10])

Muestra de las columnas de tiempo:
  Entrada programada Salida programada PRIMERA_MARCA ULTIMA_MARCA
0              07:50             15:30      06:34:03     15:24:36
1              07:50             15:30      07:45:40     17:39:38
2              07:50             14:30      07:10:36     15:00:22
3              07:50             15:30      06:57:57     15:20:38
4              07:50             15:30      07:10:51     16:06:15
5              14:00             22:00      13:30:30     22:00:41
6              07:00             13:00      06:54:03     06:54:03
7              07:50             15:30      09:02:47     14:01:18
8              07:50             15:30      06:28:35     14:25:36
9              07:50             15:30      07:34:47     15:06:06

Tipos de datos:
Entrada programada    object
Salida programada     object
PRIMERA_MARCA         object
ULTIMA_MARCA          object
dtype: object

Ejemplos únicos de 'Entrada programada':
['07:50' '14:00' '07:00' '05:50' '08:00' '06:00' '

In [14]:
#%% Exportar archivo en Excel con UTF-8
import datetime
now = datetime.datetime.now()
timestamp_str = now.strftime("%Y-%m-%d_%H-%M-%S") 
titulo = "df_rellenar_con_observaciones"
nombre_archivo = f'C:/Users/bgacitua/Desktop/Reportes Corrección de Marcas/Planillas_Python_Spyder/{titulo}_{timestamp_str}.xlsx'

# Exportar a Excel con formato UTF-8
df_rellenar.to_excel(nombre_archivo, sheet_name='Reporte_Marcas', index=False, engine='openpyxl')

print(f"\n¡Reporte exportado exitosamente en formato Excel a '{nombre_archivo}'!")
print(f"Registros exportados: {len(df_rellenar)}")
print(f"Columnas incluidas: {list(df_rellenar.columns)}")
print(f"Registros con observación de turno: {df_rellenar['Observación de turno'].notna().sum()}")


¡Reporte exportado exitosamente en formato Excel a 'C:/Users/bgacitua/Desktop/Reportes Corrección de Marcas/Planillas_Python_Spyder/df_rellenar_con_observaciones_2025-08-12_07-56-58.xlsx'!
Registros exportados: 85
Columnas incluidas: ['RUT', 'Nombre', 'Unidad', 'Puesto', 'Fecha', 'Tipo de turno', 'Entrada programada', 'Salida programada', 'Marca de entrada', 'Marca de salida', 'DV', 'Busqueda', 'KEY_IN', 'KEY_OUT', 'NO_KEY', 'PRIMERA_MARCA', 'ULTIMA_MARCA', 'Marca de entrada Generados', 'Marca de salida Generados', 'Observación de turno']
Registros con observación de turno: 6


In [15]:
# %%
# --- 4. Cerrar la Conexión (al finalizar todas las operaciones) ---
if 'cnxn' in locals() and cnxn: # Verificar si 'cnxn' existe y no es None
    try:
        cnxn.close()
        print("\nConexión a SQL Server cerrada exitosamente.")
    except Exception as e:
        print(f"Error al cerrar la conexión: {e}")
else:
    print("La conexión no fue establecida o ya está cerrada.")


Conexión a SQL Server cerrada exitosamente.


In [16]:
#Importar dataframe de marcas no asignadas
path_marcas = r"C:\Users\bgacitua\Downloads\yB4yleh2_reporte_marca_sin_turno.xlsx"
reporte_marcas_no_asignadas = pd.read_excel(path_marcas)

print("DataFrame original:")
print(reporte_marcas_no_asignadas.head())

# Transformar valores "-" en NaN en la columna "RUT Trabajador"
print(f"\nAntes de la transformación:")
print(f"Valores únicos en 'RUT Trabajador': {reporte_marcas_no_asignadas['RUT Trabajador'].unique()[:10]}")
print(f"Cantidad de valores '-': {(reporte_marcas_no_asignadas['RUT Trabajador'] == '-').sum()}")

# Reemplazar "-" con NaN
reporte_marcas_no_asignadas['RUT Trabajador'] = reporte_marcas_no_asignadas['RUT Trabajador'].replace('-', np.nan)

print(f"\nDespués de la transformación:")
print(f"Valores únicos en 'RUT Trabajador': {reporte_marcas_no_asignadas['RUT Trabajador'].unique()[:10]}")
print(f"Cantidad de valores NaN: {reporte_marcas_no_asignadas['RUT Trabajador'].isna().sum()}")

print("\nDataFrame después de la transformación:")
display(reporte_marcas_no_asignadas.head())

# Eliminar filas con valores NaN en la columna "RUT Trabajador"
print(f"\nAntes de eliminar filas con NaN:")
print(f"Total de registros: {len(reporte_marcas_no_asignadas)}")
print(f"Registros con NaN en 'RUT Trabajador': {reporte_marcas_no_asignadas['RUT Trabajador'].isna().sum()}")

# Eliminar filas donde 'RUT Trabajador' es NaN
reporte_marcas_no_asignadas = reporte_marcas_no_asignadas.dropna(subset=['RUT Trabajador'])

print(f"\nDespués de eliminar filas con NaN:")
print(f"Total de registros: {len(reporte_marcas_no_asignadas)}")
print(f"Registros con NaN en 'RUT Trabajador': {reporte_marcas_no_asignadas['RUT Trabajador'].isna().sum()}")

print("\nDataFrame final limpio:")
display(reporte_marcas_no_asignadas.head())



DataFrame original:
  RUT Trabajador                    Nombre  \
0              -                             
1              -                             
2     21889457-7  Fernando Ramos Velásquez   
3     18756037-3         José Cortés Marín   
4              -                             

                                    Unidad              Puesto Fecha Marca  \
0                                      NaN                 NaN  04/08/2025   
1                                      NaN                 NaN  04/08/2025   
2  Bodega Recepción, Despacho y Transporte  Ayudante De Bodega  04/08/2025   
3              Ingeniería y Medio Ambiente            Jefatura  04/08/2025   
4                                      NaN                 NaN  04/08/2025   

  Hora Marca Sentido Marca  
0   05:40:25             E  
1   10:59:51             E  
2   16:50:41             E  
3   17:09:41             S  
4   16:56:32             S  

Antes de la transformación:
Valores únicos en 'RUT Trabajad

,RUT Trabajador,Nombre,Unidad,Puesto,Fecha Marca,Hora Marca,Sentido Marca
0,NaN,,NaN,NaN,04/08/2025,05:40:25,E
1,NaN,,NaN,NaN,04/08/2025,10:59:51,E
2,21889457-7,Fernando Ramos Velásquez,"Bodega Recepción, Despacho y Transporte",Ayudante De Bodega,04/08/2025,16:50:41,E
3,18756037-3,José Cortés Marín,Ingeniería y Medio Ambiente,Jefatura,04/08/2025,17:09:41,S
4,NaN,,NaN,NaN,04/08/2025,16:56:32,S



Antes de eliminar filas con NaN:
Total de registros: 197
Registros con NaN en 'RUT Trabajador': 100

Después de eliminar filas con NaN:
Total de registros: 97
Registros con NaN en 'RUT Trabajador': 0

DataFrame final limpio:


,RUT Trabajador,Nombre,Unidad,Puesto,Fecha Marca,Hora Marca,Sentido Marca
2,21889457-7,Fernando Ramos Velásquez,"Bodega Recepción, Despacho y Transporte",Ayudante De Bodega,04/08/2025,16:50:41,E
3,18756037-3,José Cortés Marín,Ingeniería y Medio Ambiente,Jefatura,04/08/2025,17:09:41,S
5,18284842-5,Felipe López Villanueva,Esencias y Env. Sabores,Operario,04/08/2025,16:53:22,S
6,26954249-7,Maria Garcia Sivada,Sabores Bebidas y Dulces,Asistente,04/08/2025,17:01:19,E
7,14156981-3,David Tapia Lobos,Informática,Desarrollador,04/08/2025,17:01:34,E


In [17]:
reporte_marcas_no_asignadas.info()

<class 'pandas.core.frame.DataFrame'>
Index: 97 entries, 2 to 196
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   RUT Trabajador  97 non-null     object
 1   Nombre          97 non-null     object
 2   Unidad          97 non-null     object
 3   Puesto          97 non-null     object
 4   Fecha Marca     97 non-null     object
 5   Hora Marca      97 non-null     object
 6   Sentido Marca   97 non-null     object
dtypes: object(7)
memory usage: 6.1+ KB


In [18]:
# Separar la columna "RUT Trabajador" por el delimitador "-"
print(f"\n🔧 SEPARANDO RUT Y DÍGITO VERIFICADOR:")
print(f"Procesando columna 'RUT Trabajador'...")

# Verificar si contiene el delimitador "-"
tiene_delimitador = reporte_marcas_no_asignadas['RUT Trabajador'].astype(str).str.contains('-', na=False)

print(f"Registros con delimitador '-': {tiene_delimitador.sum()}")
print(f"Total de registros: {len(reporte_marcas_no_asignadas)}")

if tiene_delimitador.any():
    # Separar RUT y DV usando el delimitador "-"
    rut_split = reporte_marcas_no_asignadas['RUT Trabajador'].astype(str).str.split('-', expand=True)
    
    # Actualizar la columna 'RUT Trabajador' con solo la parte numérica
    reporte_marcas_no_asignadas['RUT Trabajador'] = rut_split[0]
    
    # Crear la columna 'DV' con el dígito verificador
    reporte_marcas_no_asignadas['DV'] = rut_split[1].fillna('')
    
    print(f"✅ Separación completada:")
    print(f"   • Registros procesados: {tiene_delimitador.sum()}")
    print(f"   • RUT Trabajador: solo números")
    print(f"   • DV: nueva columna creada")
    
    # Mostrar ejemplos de la separación
    print(f"\n📋 Ejemplos de separación:")
    ejemplos_separacion = reporte_marcas_no_asignadas[['RUT Trabajador', 'DV']].head()
    display(ejemplos_separacion)
    
else:
    # Si no hay delimitador, crear columna DV vacía
    reporte_marcas_no_asignadas['DV'] = ''
    print("ℹ️  No se encontraron delimitadores '-' en la columna 'RUT Trabajador'")

# Convertir RUT Trabajador a numérico para mejor manejo
try:
    reporte_marcas_no_asignadas['RUT Trabajador'] = pd.to_numeric(
        reporte_marcas_no_asignadas['RUT Trabajador'], errors='coerce'
    )
    print(f"✅ Columna 'RUT Trabajador' convertida a tipo numérico")
except Exception as e:
    print(f"⚠️  Advertencia al convertir 'RUT Trabajador' a numérico: {e}")

print(f"\n📊 RESULTADO FINAL:")
print(f"DataFrame con RUT y DV separados:")
display(reporte_marcas_no_asignadas.head())

# Convertir formato de fecha de dd/mm/yyyy a yyyy-mm-dd
print(f"\n📅 CONVIRTIENDO FORMATO DE FECHA:")
print(f"Procesando columna 'Fecha Marca' desde dd/mm/yyyy hacia yyyy-mm-dd...")

# Mostrar formato actual
print(f"\nAntes de la conversión:")
print(f"Tipo de dato: {reporte_marcas_no_asignadas['Fecha Marca'].dtype}")
print(f"Ejemplos de fechas actuales:")
print(reporte_marcas_no_asignadas['Fecha Marca'].head())

try:
    # Convertir de dd/mm/yyyy a datetime
    reporte_marcas_no_asignadas['Fecha Marca'] = pd.to_datetime(
        reporte_marcas_no_asignadas['Fecha Marca'], 
        format='%d/%m/%Y', 
        errors='coerce'
    )
    
    # Convertir a formato yyyy-mm-dd (string)
    reporte_marcas_no_asignadas['Fecha Marca'] = reporte_marcas_no_asignadas['Fecha Marca'].dt.strftime('%Y-%m-%d')
    
    print(f"\n✅ Conversión completada exitosamente")
    print(f"Nuevo tipo de dato: {reporte_marcas_no_asignadas['Fecha Marca'].dtype}")
    print(f"Ejemplos de fechas convertidas:")
    print(reporte_marcas_no_asignadas['Fecha Marca'].head())
    
    # Verificar si hay fechas que no pudieron convertirse
    fechas_nulas = reporte_marcas_no_asignadas['Fecha Marca'].isna().sum()
    if fechas_nulas > 0:
        print(f"⚠️  {fechas_nulas} fechas no pudieron convertirse y quedaron como NaN")
    else:
        print(f"🎉 Todas las fechas se convirtieron correctamente")
        
except Exception as e:
    print(f"❌ Error al convertir fechas: {str(e)}")
    print(f"💡 Verifica que las fechas estén en formato dd/mm/yyyy")

# Mostrar muestra final del DataFrame
print(f"\n🔍 MUESTRA FINAL DEL DATAFRAME:")
columnas_clave = ['RUT Trabajador', 'DV', 'Fecha Marca', 'Hora Marca']
columnas_disponibles = [col for col in columnas_clave if col in reporte_marcas_no_asignadas.columns]
display(reporte_marcas_no_asignadas[columnas_disponibles].head())

print(f"\n📋 RESUMEN DEL PROCESAMIENTO:")
print(f"   • Total de registros: {len(reporte_marcas_no_asignadas)}")
print(f"   • RUT Trabajador: formato numérico")
print(f"   • DV: dígito verificador separado")
print(f"   • Fecha Marca: formato yyyy-mm-dd")
print(f"   • Datos listos para análisis ✅")


🔧 SEPARANDO RUT Y DÍGITO VERIFICADOR:
Procesando columna 'RUT Trabajador'...
Registros con delimitador '-': 97
Total de registros: 97
✅ Separación completada:
   • Registros procesados: 97
   • RUT Trabajador: solo números
   • DV: nueva columna creada

📋 Ejemplos de separación:


,RUT Trabajador,DV
2,21889457,7
3,18756037,3
5,18284842,5
6,26954249,7
7,14156981,3


✅ Columna 'RUT Trabajador' convertida a tipo numérico

📊 RESULTADO FINAL:
DataFrame con RUT y DV separados:


,RUT Trabajador,Nombre,Unidad,Puesto,Fecha Marca,Hora Marca,Sentido Marca,DV
2,21889457,Fernando Ramos Velásquez,"Bodega Recepción, Despacho y Transporte",Ayudante De Bodega,04/08/2025,16:50:41,E,7
3,18756037,José Cortés Marín,Ingeniería y Medio Ambiente,Jefatura,04/08/2025,17:09:41,S,3
5,18284842,Felipe López Villanueva,Esencias y Env. Sabores,Operario,04/08/2025,16:53:22,S,5
6,26954249,Maria Garcia Sivada,Sabores Bebidas y Dulces,Asistente,04/08/2025,17:01:19,E,7
7,14156981,David Tapia Lobos,Informática,Desarrollador,04/08/2025,17:01:34,E,3



📅 CONVIRTIENDO FORMATO DE FECHA:
Procesando columna 'Fecha Marca' desde dd/mm/yyyy hacia yyyy-mm-dd...

Antes de la conversión:
Tipo de dato: object
Ejemplos de fechas actuales:
2    04/08/2025
3    04/08/2025
5    04/08/2025
6    04/08/2025
7    04/08/2025
Name: Fecha Marca, dtype: object

✅ Conversión completada exitosamente
Nuevo tipo de dato: object
Ejemplos de fechas convertidas:
2    2025-08-04
3    2025-08-04
5    2025-08-04
6    2025-08-04
7    2025-08-04
Name: Fecha Marca, dtype: object
🎉 Todas las fechas se convirtieron correctamente

🔍 MUESTRA FINAL DEL DATAFRAME:


,RUT Trabajador,DV,Fecha Marca,Hora Marca
2,21889457,7,2025-08-04,16:50:41
3,18756037,3,2025-08-04,17:09:41
5,18284842,5,2025-08-04,16:53:22
6,26954249,7,2025-08-04,17:01:19
7,14156981,3,2025-08-04,17:01:34



📋 RESUMEN DEL PROCESAMIENTO:
   • Total de registros: 97
   • RUT Trabajador: formato numérico
   • DV: dígito verificador separado
   • Fecha Marca: formato yyyy-mm-dd
   • Datos listos para análisis ✅
